In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import higlass
import higlass.client as hgc
import higlass.tilesets as hti

ts1 = hti.cooler('../data/Dixon2012-J1-NcoI-R1-filtered.100kb.multires.cool')

tr1 = hgc.Track('top-axis')
tr2 = hgc.Track('heatmap', tileset=ts1, height=100)
#higlass.display([hgc.View([tr1, tr2])])()

In [8]:
import higlass.tilesets as hfti

ts = hfti.mmatrix('../data/int_matrices.hdf5')

tr1 = hgc.Track('top-axis')
tr2 = hgc.Track('heatmap', tileset=ts, height=200)

(server, display) = higlass.display([hgc.View([tr1, tr2])])

display

terminating: Y-AitNFTThS05r9g6fcdLg
self.tilesets: [<higlass.tilesets.Tileset object at 0x1122c2f60>]
 * Serving Flask app "higlass.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
info: {'bins_per_dimension': 256, 'max_pos': [180, 180], 'max_width': 360, 'max_zoom': 6, 'min_pos': [-180, -180], 'mirror_tiles': 'false'}


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Zfr05F-BToCGVLEuo-UeKg', 'tracks': {'top': [{'ty…

In [33]:
height=200

tr0 = hgc.Track('top-axis')
tr1 = hgc.Track('osm-tiles', height=height)

view1 = hgc.View([tr0, tr1], width=6)
ts = hfti.mmatrix('../data/int_matrices.hdf5')

tr1 = hgc.Track('top-axis')
tr2 = hgc.Track('heatmap', tileset=ts, height=height)
view2 = hgc.View([tr1, tr2], x=6, y=0, width=6)

(server, display) = higlass.display([view1, view2], 
                                    location_sync=[view1, view2],
                                    zoom_sync = [view1, view2])
display

terminating: G_rkOeq_TA--hdF7DZ_NqA
self.tilesets: [<higlass.tilesets.Tileset object at 0x11271ec18>]
 * Serving Flask app "higlass.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
adding: dkei5hkSSQWxl71RRmftlw
adding: Rro0VIg4RgSSOtyrEGstbQ
adding: dkei5hkSSQWxl71RRmftlw
adding: Rro0VIg4RgSSOtyrEGstbQ


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'dkei5hkSSQWxl71RRmftlw', 'tracks': {'top': [{'ty…

In [36]:
height=200

tr0 = hgc.Track('top-axis')
tr1 = hgc.Track('heatmap', tileset=ts, height=height)

view1 = hgc.View([tr0, tr1], width=6)
ts = hfti.mmatrix('../data/int_matrices.hdf5')

tr1 = hgc.Track('top-axis')
tr2 = hgc.Track('heatmap', tileset=ts, height=height)
view2 = hgc.View([tr1, tr2], x=6, y=0, width=6)

(server, display) = higlass.display([view1, view2], 
                                    zoom_sync = [view1, view2])
display

terminating: ffKQewigQEidGjIWPXLfvw
self.tilesets: [<higlass.tilesets.Tileset object at 0x11271e160>, <higlass.tilesets.Tileset object at 0x11229abe0>]
 * Serving Flask app "higlass.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
adding: IOXvvTc0RFygpPI1yUhRGQ
adding: VoFdjoThQgOByd6fv4Fpvw


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'IOXvvTc0RFygpPI1yUhRGQ', 'tracks': {'top': [{'ty…